# Libraries

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_columns', None)

# Data

## Intake data

### Data url

In [3]:
intake_url = "https://data.austintexas.gov/resource/wter-evkm.csv?$limit=500000"

### Data pull

In [4]:
df_intakes = pd.read_csv(intake_url)

### Data preview

In [5]:
df_intakes.head()

,animal_id,name,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
0,A521520,Nina,2013-10-01T07:51:00.000,2013-10-01T07:51:00.000,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
1,A664235,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
2,A664236,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
3,A664237,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
4,A664233,Stevie,2013-10-01T08:53:00.000,2013-10-01T08:53:00.000,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White


## Outcome data

### Data url

In [6]:
outcome_url = "https://data.austintexas.gov/resource/9t4d-g238.csv?$limit=500000"

### Data pull

In [7]:
df_outcomes = pd.read_csv(outcome_url)

### Data preview

In [8]:
df_outcomes.head(20)

,animal_id,date_of_birth,name,datetime,monthyear,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
0,A668305,2012-12-01,NaN,2013-12-02T00:00:00-05:00,12-2013,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Brown/Yellow
1,A673335,2012-02-22,NaN,2014-02-22T00:00:00-05:00,02-2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon,Black/Gray
2,A675999,2013-04-03,NaN,2014-04-07T00:00:00-05:00,04-2014,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Green
3,A679066,2014-04-16,NaN,2014-05-16T00:00:00-05:00,05-2014,NaN,NaN,Other,Unknown,4 weeks,Rabbit Sh,Brown
4,A680855,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
5,A680857,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
6,A680858,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
7,A680859,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
8,A680860,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
9,A680861,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black


# EDA

## Missing values in intakes

In [9]:
# Check for missing values in df_outcomes
missing_values = df_intakes.isnull().sum()
print("Missing values in df_intakes:\n", missing_values)

Missing values in df_intakes:
 animal_id               0
name                49991
datetime                0
datetime2               0
found_location          0
intake_type             0
intake_condition        0
animal_type             0
sex_upon_intake         1
age_upon_intake         0
breed                   0
color                   0
dtype: int64


## Missing values in outcomes

In [10]:
# Check for missing values in df_outcomes
missing_values = df_outcomes.isnull().sum()
print("Missing values in df_outcomes:\n", missing_values)

Missing values in df_outcomes:
 animal_id               0
date_of_birth           0
name                49784
datetime                0
monthyear               0
outcome_type           46
outcome_subtype     94115
animal_type             0
sex_upon_outcome        1
age_upon_outcome        9
breed                   0
color                   0
dtype: int64


## Unique values in color

In [11]:
df_intakes['color'].nunique()

661

In [12]:
df_outcomes['color'].nunique()

663

High cardinality exists here

## Unique values in found_location

In [13]:
df_intakes['found_location'].nunique()

70183

High cardinality exists here

## Unique values in breed

In [14]:
df_intakes['breed'].nunique()

3006

In [15]:
df_outcomes['breed'].nunique()

3010

High cardinality here too

# Preprocessing

## Fix datetime column

### Intakes

In [16]:
df_intakes['datetime'] = pd.to_datetime(df_intakes['datetime'])

In [17]:
df_intakes['monthyear'] = pd.to_datetime(df_intakes['datetime2']).dt.strftime('%m-%Y')

In [18]:
df_intakes = df_intakes.drop(columns=['datetime2'])

### Outcomes

In [19]:
df_outcomes['date_of_birth'] = pd.to_datetime(df_outcomes['date_of_birth'])
df_outcomes['monthyear'] = pd.to_datetime(df_outcomes['monthyear'], format='%m-%Y')

In [20]:
df_outcomes['datetime'] = pd.to_datetime(df_outcomes['datetime'], format='mixed', utc=True)

## Dropping null values from both dataframes

For **df_intakes**, missing names can be handled later in a binary variable, while, variables like *sex_upon_intake* will need to be omitted from the analysis.

For **df_outcomes**, missing values in age and sex will be omitted. 

In [21]:
df_intakes = df_intakes.dropna(subset=['sex_upon_intake'])

In [22]:
df_outcomes = df_outcomes.dropna(subset=['outcome_type', 'sex_upon_outcome', 'age_upon_outcome'])

## Combining both dataframes into one

### First, sort by date

In [23]:
df_intakes_sorted = df_intakes.sort_values('datetime')
df_outcomes_sorted = df_outcomes.sort_values('datetime')

### Combine both dataframes into one

In [24]:
combined_df = pd.merge(df_intakes_sorted, df_outcomes_sorted, on='animal_id', suffixes=('_intake', '_outcome'), how='inner')

## Age variable fix

In [25]:
def convert_age_to_days(age_str, name_str=None):
    """Convert age string to numeric days"""
    if pd.isna(age_str):
        return None
    
    age_str = age_str.lower()
    
    # Handle "0 years" case - check if newborn based on name containing "grams"
    if age_str.strip() == "0 years":
        if name_str and "grams" in str(name_str).lower():
            return 0  # Newborn
        else:
            return None  # Missing age data
    
    total_days = 0
    
    # Find all number-unit pairs
    patterns = [
        (r'(\d+)\s*year', 365),
        (r'(\d+)\s*month', 30),
        (r'(\d+)\s*week', 7),
        (r'(\d+)\s*day', 1)
    ]
    
    for pattern, multiplier in patterns:
        matches = re.findall(pattern, age_str)
        for match in matches:
            total_days += int(match) * multiplier
    
    return total_days if total_days > 0 else None

def convert_age_to_years(age_str, name_str=None):
    """Convert age string to numeric years (decimal)"""
    days = convert_age_to_days(age_str, name_str)
    return days / 365 if days is not None else None

In [26]:
df_intakes['age_upon_intake_days'] = df_intakes.apply(lambda row: convert_age_to_days(row['age_upon_intake'], row.get('name')), axis=1)
df_intakes['age_upon_intake_years'] = df_intakes.apply(lambda row: convert_age_to_years(row['age_upon_intake'], row.get('name')), axis=1)

df_outcomes['age_upon_outcome_days'] = df_outcomes.apply(lambda row: convert_age_to_days(row['age_upon_outcome'], row.get('name')), axis=1)
df_outcomes['age_upon_outcome_years'] = df_outcomes.apply(lambda row: convert_age_to_years(row['age_upon_outcome'], row.get('name')), axis=1)


In [27]:
print("Sample conversions:")
test_ages = ["7 years", "2 weeks", "5 months", "10 days"]
for age in test_ages:
    print(f"{age} -> {convert_age_to_days(age)} days, {convert_age_to_years(age):.2f} years")

Sample conversions:
7 years -> 2555 days, 7.00 years
2 weeks -> 14 days, 0.04 years
5 months -> 150 days, 0.41 years
10 days -> 10 days, 0.03 years


In [39]:
combined_df['age_upon_outcome_days'] = combined_df.apply(lambda row: convert_age_to_days(row['age_upon_outcome'], row.get('name')), axis=1)
combined_df['age_upon_outcome_years'] = combined_df.apply(lambda row: convert_age_to_years(row['age_upon_outcome'], row.get('name')), axis=1)

combined_df['age_upon_intake_days'] = combined_df.apply(lambda row: convert_age_to_days(row['age_upon_intake'], row.get('name')), axis=1)
combined_df['age_upon_intake_years'] = combined_df.apply(lambda row: convert_age_to_years(row['age_upon_intake'], row.get('name')), axis=1)

In [40]:
combined_df.isna().sum()

animal_id                      0
name_intake                50645
datetime_intake                0
found_location                 0
intake_type                    0
intake_condition               0
animal_type_intake             0
sex_upon_intake                0
age_upon_intake                0
breed_intake                   0
color_intake                   0
monthyear_intake               0
date_of_birth                  0
name_outcome               50645
datetime_outcome               0
monthyear_outcome              0
outcome_type                   0
outcome_subtype           133819
animal_type_outcome            0
sex_upon_outcome               0
age_upon_outcome               0
breed_outcome                  0
color_outcome                  0
age_upon_outcome_days        283
age_upon_outcome_years       283
age_upon_intake_days        1284
age_upon_intake_years       1284
dtype: int64

In [28]:
df_outcomes.isna().sum()

animal_id                     0
date_of_birth                 0
name                      49759
datetime                      0
monthyear                     0
outcome_type                  0
outcome_subtype           94070
animal_type                   0
sex_upon_outcome              0
age_upon_outcome              0
breed                         0
color                         0
age_upon_outcome_days       248
age_upon_outcome_years      248
dtype: int64

In [29]:
df_intakes.isnull().sum()

animal_id                    0
name                     49991
datetime                     0
found_location               0
intake_type                  0
intake_condition             0
animal_type                  0
sex_upon_intake              0
age_upon_intake              0
breed                        0
color                        0
monthyear                    0
age_upon_intake_days      1236
age_upon_intake_years     1236
dtype: int64

### Dropping null values for age upon intake & age upon outcome

In [30]:
df_intakes = df_intakes.dropna(subset=['age_upon_intake_days'])

In [31]:
df_outcomes = df_outcomes.dropna(subset=['age_upon_outcome_days'])

### Dropping age upon intake and age upon outcome columns

In [32]:
df_intakes = df_intakes.drop(columns=['age_upon_intake'])
df_outcomes = df_outcomes.drop(columns=['age_upon_outcome'])

In [ ]:
# Null values in 

# 1. Check for null values in the dataset
print(df_intakes.isnull().sum())

# 2. Check for null values in the dataset
print(df_outcomes.isnull().sum())

## Breed variable fix

In [33]:
def engineer_breed_features(df, breed_column='breed'):
    """Extract meaningful features from breed information"""
    
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Animal type already exists, so skipping that classification
    
    # Mixed breed indicators
    df['is_mixed'] = df[breed_column].str.contains('Mix|/', case=False, na=False)
    df['num_breeds'] = df[breed_column].str.count('/') + 1
    df['num_breeds'] = df['num_breeds'].where(df['is_mixed'], 1)
    
    # Size categories (based on common breed patterns)
    toy_breeds = ['Chihuahua', 'Yorkshire', 'Toy', 'Maltese', 'Pomeranian', 'Papillon', 'Miniature']
    small_breeds = ['Terrier', 'Beagle', 'Cocker', 'Dachshund', 'Corgi', 'Pug', 'Shih Tzu']
    large_breeds = ['Retriever', 'Shepherd', 'Mastiff', 'Great Dane', 'Rottweiler', 'Great Pyrenees', 'Newfoundland']
    
    def categorize_size(breed_str):
        if pd.isna(breed_str):
            return 'Unknown'
        breed_str = str(breed_str)
        if any(toy in breed_str for toy in toy_breeds):
            return 'Toy'
        elif any(large in breed_str for large in large_breeds):
            return 'Large'
        elif any(small in breed_str for small in small_breeds):
            return 'Small'
        else:
            return 'Medium'
    
    df['size_category'] = df[breed_column].apply(categorize_size)
    
    # Working/sport groups
    working_breeds = ['Shepherd', 'Cattle Dog', 'Border Collie', 'Australian Kelpie', 'Husky', 'Malamute']
    sporting_breeds = ['Retriever', 'Pointer', 'Spaniel', 'Setter', 'Vizsla']
    terrier_breeds = ['Terrier']
    
    df['is_working'] = df[breed_column].str.contains('|'.join(working_breeds), case=False, na=False)
    df['is_sporting'] = df[breed_column].str.contains('|'.join(sporting_breeds), case=False, na=False)
    df['is_terrier'] = df[breed_column].str.contains('Terrier', case=False, na=False)
    
    # Popular breed indicators
    popular_dog_breeds = ['Labrador', 'Golden Retriever', 'Pit Bull', 'Chihuahua', 'Beagle', 'German Shepherd']
    popular_cat_breeds = ['Domestic Shorthair', 'Domestic Longhair', 'Siamese']
    
    df['is_popular_dog'] = df[breed_column].str.contains('|'.join(popular_dog_breeds), case=False, na=False)
    df['is_popular_cat'] = df[breed_column].str.contains('|'.join(popular_cat_breeds), case=False, na=False)
    
    # Exotic/unusual animals
    exotic_animals = ['Snake', 'Lizard', 'Turtle', 'Bird', 'Rabbit', 'Ferret', 'Pig', 'Goat', 'Chicken']
    df['is_exotic'] = df[breed_column].str.contains('|'.join(exotic_animals), case=False, na=False)
    
    # Primary breed (for mixed breeds, take the first one)
    def extract_primary_breed(breed_str):
        if pd.isna(breed_str):
            return None
        breed_str = str(breed_str)
        if '/' in breed_str:
            return breed_str.split('/')[0].strip()
        elif 'Mix' in breed_str:
            return breed_str.replace(' Mix', '').strip()
        else:
            return breed_str.strip()
    
    df['primary_breed'] = df[breed_column].apply(extract_primary_breed)
    
    # Breed complexity score (more complex = more mixed)
    df['breed_complexity'] = df['num_breeds'] + df['is_mixed'].astype(int)
    
    return df

In [43]:
combined_df = engineer_breed_features(combined_df, 'breed_outcome')

In [44]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221168 entries, 0 to 221167
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   animal_id               221168 non-null  object             
 1   name_intake             170523 non-null  object             
 2   datetime_intake         221168 non-null  datetime64[ns]     
 3   found_location          221168 non-null  object             
 4   intake_type             221168 non-null  object             
 5   intake_condition        221168 non-null  object             
 6   animal_type_intake      221168 non-null  object             
 7   sex_upon_intake         221168 non-null  object             
 8   age_upon_intake         221168 non-null  object             
 9   breed_intake            221168 non-null  object             
 10  color_intake            221168 non-null  object             
 11  monthyear_intake        22

In [45]:
combined_df = combined_df.drop(columns=['breed_intake','breed_outcome'])

## Has name column

### Fix cases with weight in place of name

Some newborn animals have their weight put into the name column. Let's fix this

In [46]:
# Replace weight entries with null values
combined_df['name_intake'] = combined_df['name_intake'].replace(r'^\d+\s*grams?$', pd.NA, regex=True)
combined_df['name_outcome'] = combined_df['name_outcome'].replace(r'^\d+\s*grams?$', pd.NA, regex=True)

### If name at outcome is not null, has_name will be 1, 0 otherwise

In [47]:
combined_df['has_name'] = combined_df['name_outcome'].notnull().astype(int)

## Keeping only Dogs and Cats

Less than 6% if instances are not dogs or cats. Thus, for this model, dropping those instances makes sense.

In [49]:
combined_df = combined_df[combined_df['animal_type_outcome'].isin(['Dog', 'Cat'])]

## Fix found_location column to extract feature

The good thing here is that the location an animal is found is noted down, but, due to high cardinality, this feature also needs to be engineered. Capturing the jurisdiction, and area related information is going to be engineered.

In [56]:
def engineer_location_features(df, location_column='found_location'):
    """Extract meaningful features from found_location"""
    
    df = df.copy()
    
    # Extract jurisdiction/city from parentheses or end of string
    def extract_jurisdiction(location_str):
        if pd.isna(location_str):
            return None
        location_str = str(location_str).strip()
        
        # Pattern for (TX) or (County)
        paren_match = re.search(r'\(([^)]+)\)', location_str)
        if paren_match:
            return paren_match.group(1)
        
        # For cases like "Manor(TX)" without space
        no_space_match = re.search(r'([A-Za-z]+)\([^)]+\)$', location_str)
        if no_space_match:
            return no_space_match.group(1)
        
        return None
    
    # Extract city/area name
    def extract_city(location_str):
        if pd.isna(location_str):
            return None
        location_str = str(location_str).strip()
        
        # Handle "Outside jurisdiction"
        if 'outside jurisdiction' in location_str.lower():
            return 'Outside Jurisdiction'
        
        # Extract city before " in " or before parentheses
        if ' in ' in location_str:
            parts = location_str.split(' in ')
            if len(parts) > 1:
                city_part = parts[1].split('(')[0].strip()
                return city_part
        
        # For direct formats like "Austin (TX)" or "Manor(TX)"
        city_match = re.search(r'^([A-Za-z\s]+)(?:\s*\(|$)', location_str)
        if city_match:
            return city_match.group(1).strip()
        
        return location_str
    
    # Apply extractions
    df['jurisdiction'] = df[location_column].apply(extract_jurisdiction)
    df['city_area'] = df[location_column].apply(extract_city)
    
    # Austin area classifications
    austin_areas = ['Austin', 'Travis', 'Manor', 'Pflugerville', 'Cedar Park', 'Round Rock', 
                   'Lakeway', 'Bee Cave', 'West Lake Hills', 'Rollingwood', 'Sunset Valley']
    
    surrounding_areas = ['Williamson', 'Hays', 'Bastrop', 'Caldwell', 'Georgetown', 
                        'Leander', 'Cedar Creek', 'Elgin', 'Dripping Springs']
    
    df['is_austin_metro'] = df['city_area'].isin(austin_areas)
    df['is_surrounding_area'] = df['city_area'].isin(surrounding_areas)
    df['is_outside_jurisdiction'] = df['city_area'] == 'Outside Jurisdiction'
    
    # Core Austin vs suburbs
    df['is_core_austin'] = df['city_area'] == 'Austin'
    df['is_travis_county'] = df['city_area'].isin(['Austin', 'Travis', 'Manor'])
    
    # Distance categories (approximate)
    def categorize_distance(city):
        if pd.isna(city):
            return 'Unknown'
        if city in ['Austin', 'Travis']:
            return 'Core'
        elif city in ['Manor', 'Pflugerville', 'West Lake Hills', 'Rollingwood', 'Sunset Valley']:
            return 'Close Suburbs'
        elif city in ['Cedar Park', 'Round Rock', 'Lakeway', 'Bee Cave']:
            return 'Far Suburbs'
        elif city == 'Outside Jurisdiction':
            return 'Outside'
        else:
            return 'Other'
    
    df['distance_category'] = df['city_area'].apply(categorize_distance)
    
    # Has specific address (contains street info)
    # df['has_address_detail'] = df[location_column].str.contains(r'\d+\s+\w+', na=False)
    
    return df

In [57]:
combined_df = engineer_location_features(combined_df, 'found_location')

In [58]:
def print_location_summary(df):
    """Print summary of location features"""
    print("Location Feature Summary:")
    print(f"Core Austin: {df['is_core_austin'].sum()}")
    print(f"Travis County: {df['is_travis_county'].sum()}")
    print(f"Austin Metro: {df['is_austin_metro'].sum()}")
    print(f"Outside Jurisdiction: {df['is_outside_jurisdiction'].sum()}")
    print("\nDistance categories:")
    print(df['distance_category'].value_counts())

In [59]:
print_location_summary(combined_df)

Location Feature Summary:
Core Austin: 172768
Travis County: 199144
Austin Metro: 202672
Outside Jurisdiction: 3996

Distance categories:
distance_category
Core             195052
Close Suburbs      6814
Other              4560
Outside            3996
Far Suburbs         806
Name: count, dtype: int64


## Duplicate case fix

## Is fixed column

## Create target variable **Is_Adopted**

## Drop *outcome_subtype* 